# Importations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("../raw_data/data_final_indé_small.csv")

## Présentation de la donnée

In [3]:
data

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,english,price,has_a_website,nb_review,owner_estimated,rating,popularity,score_descriptif,average_playtime,median_playtime
0,4290,RACE: Caterham Expansion,racing simulation relaxing realistic real_time...,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,0.015822,...,1,3.99,1,6,1248.0,424.233333,517.758333,4,0,0
1,4900,Zen of Sudoku,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,0.015666,0.015666,...,1,0.00,1,139,20000.0,22124.365000,23631.392500,3,0,0
2,7340,Azada,casual puzzle 1980s relaxing realistic,0.015666,0.307211,0.015666,0.015666,0.015666,0.015666,0.015666,...,1,6.99,1,12,2496.0,7.012399,9.197810,1,0,0
3,9960,Haunted House™,adventure 1980s relaxing realistic real_time_w...,0.015546,0.015546,0.015546,0.015546,0.015546,0.015546,0.015546,...,1,4.79,1,22,4576.0,215.087500,263.378125,3,0,0
4,9970,Star Raiders,action space 1980s realistic real_time_with_pause,0.016158,0.016158,0.016158,0.016158,0.016158,0.016158,0.016158,...,1,6.99,1,29,6032.0,209.787500,263.028125,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18524,1065230,Room of Pandora,adventure casual puzzle 1980s relaxing,0.015694,0.701810,0.015694,0.015694,0.015694,0.015694,0.015694,...,1,2.09,0,3,624.0,3.000000,3.000000,1,0,0
18525,1065570,Cyber Gun,action adventure fps 3d_platformer fast_paced,0.016478,0.016478,0.519926,0.016478,0.183470,0.016478,0.016478,...,1,1.69,0,9,1872.0,13.366667,16.816667,3,0,0
18526,1065650,Super Star Blast,action casual 1980s relaxing realistic,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,...,1,3.99,1,1,208.0,4.600000,10.450000,5,0,0
18527,1066700,New Yankee 7: Deer Hunters,casual adventure 1980s relaxing realistic,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,...,1,5.19,1,2,416.0,23.463964,29.301802,2,0,0


## Sélection de X et Y

In [4]:
X = data.drop(columns=["steam_appid",
                        "name",
                       "top_5_tags",
                       "nb_review",
                       "owner_estimated",
                       "rating",
                       "topic_7",
                       "topic_11",
                       "popularity",
                      "average_playtime",
                    "median_playtime"])

y = data['owner_estimated']

## Train test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Scale de la donnée

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[-0.29704208, -0.2520625 , -0.24971488, ..., -0.72123791,
         0.79424879, -1.25339978],
       [-0.28972277, -0.24200564, -0.2407121 , ..., -0.05056938,
        -1.25905134,  0.69098295],
       [-0.2934679 , -0.24715152, -0.24531862, ...,  0.22317287,
        -1.25905134,  0.69098295],
       ...,
       [-0.29761512, -0.25284987, -0.25041972, ..., -0.71971711,
         0.79424879, -0.08677014],
       [-0.29396022, -0.24782799, -0.24592419, ...,  0.8923206 ,
         0.79424879,  1.85761258],
       [ 4.66534747, -0.24766061, -0.24577436, ..., -0.52201438,
        -1.25905134, -1.25339978]])

## Entrainement du modèle avec Catboost

In [7]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(custom_metric=['RMSE','R2'])
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_test)

Learning rate set to 0.06138
0:	learn: 6326.4241977	total: 61.2ms	remaining: 1m 1s
1:	learn: 6278.3268680	total: 64.4ms	remaining: 32.1s
2:	learn: 6233.8183915	total: 67.3ms	remaining: 22.4s
3:	learn: 6195.1756927	total: 69.9ms	remaining: 17.4s
4:	learn: 6161.0822501	total: 72.9ms	remaining: 14.5s
5:	learn: 6128.6680833	total: 75.2ms	remaining: 12.5s
6:	learn: 6098.2242337	total: 77.6ms	remaining: 11s
7:	learn: 6073.1697376	total: 80.3ms	remaining: 9.95s
8:	learn: 6047.4760093	total: 82.9ms	remaining: 9.13s
9:	learn: 6025.3298418	total: 85.3ms	remaining: 8.44s
10:	learn: 6004.6124173	total: 88.7ms	remaining: 7.97s
11:	learn: 5983.1912458	total: 91ms	remaining: 7.49s
12:	learn: 5965.7914212	total: 93.4ms	remaining: 7.09s
13:	learn: 5948.7352256	total: 95.9ms	remaining: 6.75s
14:	learn: 5933.8884164	total: 98.2ms	remaining: 6.45s
15:	learn: 5919.5130862	total: 101ms	remaining: 6.2s
16:	learn: 5903.5764316	total: 104ms	remaining: 5.99s
17:	learn: 5890.7775895	total: 106ms	remaining: 5.79s

209:	learn: 5398.5177962	total: 588ms	remaining: 2.21s
210:	learn: 5396.9799409	total: 590ms	remaining: 2.21s
211:	learn: 5395.5513040	total: 593ms	remaining: 2.2s
212:	learn: 5394.0741065	total: 595ms	remaining: 2.2s
213:	learn: 5392.6094800	total: 597ms	remaining: 2.19s
214:	learn: 5390.9731662	total: 600ms	remaining: 2.19s
215:	learn: 5389.2674109	total: 603ms	remaining: 2.19s
216:	learn: 5387.5130421	total: 605ms	remaining: 2.18s
217:	learn: 5386.5363663	total: 608ms	remaining: 2.18s
218:	learn: 5384.8115504	total: 611ms	remaining: 2.18s
219:	learn: 5383.7149116	total: 613ms	remaining: 2.17s
220:	learn: 5382.3077945	total: 616ms	remaining: 2.17s
221:	learn: 5380.2736054	total: 618ms	remaining: 2.17s
222:	learn: 5379.0306795	total: 621ms	remaining: 2.16s
223:	learn: 5377.3801418	total: 624ms	remaining: 2.16s
224:	learn: 5376.1246961	total: 626ms	remaining: 2.15s
225:	learn: 5374.3621005	total: 628ms	remaining: 2.15s
226:	learn: 5372.8921676	total: 630ms	remaining: 2.15s
227:	learn: 

365:	learn: 5205.9405328	total: 977ms	remaining: 1.69s
366:	learn: 5204.9989034	total: 980ms	remaining: 1.69s
367:	learn: 5204.1526440	total: 983ms	remaining: 1.69s
368:	learn: 5203.2022848	total: 985ms	remaining: 1.69s
369:	learn: 5202.0542809	total: 988ms	remaining: 1.68s
370:	learn: 5201.5171924	total: 991ms	remaining: 1.68s
371:	learn: 5200.4556608	total: 993ms	remaining: 1.68s
372:	learn: 5199.8370801	total: 995ms	remaining: 1.67s
373:	learn: 5199.1343641	total: 998ms	remaining: 1.67s
374:	learn: 5198.3797033	total: 1s	remaining: 1.67s
375:	learn: 5197.2704592	total: 1s	remaining: 1.66s
376:	learn: 5195.7398781	total: 1s	remaining: 1.66s
377:	learn: 5194.6279616	total: 1.01s	remaining: 1.66s
378:	learn: 5193.0582098	total: 1.01s	remaining: 1.65s
379:	learn: 5192.0399527	total: 1.01s	remaining: 1.65s
380:	learn: 5190.9288618	total: 1.01s	remaining: 1.65s
381:	learn: 5188.5757684	total: 1.02s	remaining: 1.65s
382:	learn: 5187.4125028	total: 1.02s	remaining: 1.64s
383:	learn: 5186.21

592:	learn: 4981.2830813	total: 1.55s	remaining: 1.06s
593:	learn: 4980.3872501	total: 1.55s	remaining: 1.06s
594:	learn: 4979.5243896	total: 1.55s	remaining: 1.06s
595:	learn: 4978.9766775	total: 1.55s	remaining: 1.05s
596:	learn: 4978.1087833	total: 1.56s	remaining: 1.05s
597:	learn: 4977.4553589	total: 1.56s	remaining: 1.05s
598:	learn: 4976.8691300	total: 1.56s	remaining: 1.04s
599:	learn: 4975.9837232	total: 1.56s	remaining: 1.04s
600:	learn: 4974.8091597	total: 1.57s	remaining: 1.04s
601:	learn: 4974.0057054	total: 1.57s	remaining: 1.04s
602:	learn: 4973.1931984	total: 1.57s	remaining: 1.03s
603:	learn: 4972.5282315	total: 1.57s	remaining: 1.03s
604:	learn: 4972.1667365	total: 1.58s	remaining: 1.03s
605:	learn: 4971.3389354	total: 1.58s	remaining: 1.03s
606:	learn: 4970.5598503	total: 1.58s	remaining: 1.02s
607:	learn: 4969.8421296	total: 1.58s	remaining: 1.02s
608:	learn: 4969.2325413	total: 1.59s	remaining: 1.02s
609:	learn: 4968.2966997	total: 1.59s	remaining: 1.01s
610:	learn

753:	learn: 4849.7602039	total: 1.94s	remaining: 632ms
754:	learn: 4848.7369602	total: 1.94s	remaining: 630ms
755:	learn: 4847.6576154	total: 1.94s	remaining: 627ms
756:	learn: 4846.9646895	total: 1.95s	remaining: 625ms
757:	learn: 4846.4516292	total: 1.95s	remaining: 622ms
758:	learn: 4845.9629057	total: 1.95s	remaining: 619ms
759:	learn: 4845.3397863	total: 1.95s	remaining: 617ms
760:	learn: 4844.8554610	total: 1.96s	remaining: 614ms
761:	learn: 4844.3046851	total: 1.96s	remaining: 612ms
762:	learn: 4843.7548969	total: 1.96s	remaining: 609ms
763:	learn: 4842.9502674	total: 1.96s	remaining: 606ms
764:	learn: 4842.2147677	total: 1.96s	remaining: 604ms
765:	learn: 4841.4038629	total: 1.97s	remaining: 601ms
766:	learn: 4840.6522238	total: 1.97s	remaining: 598ms
767:	learn: 4839.9621252	total: 1.97s	remaining: 596ms
768:	learn: 4839.1863659	total: 1.97s	remaining: 593ms
769:	learn: 4838.7337769	total: 1.98s	remaining: 590ms
770:	learn: 4837.7162728	total: 1.98s	remaining: 588ms
771:	learn

967:	learn: 4695.0127216	total: 2.51s	remaining: 83ms
968:	learn: 4694.0717765	total: 2.51s	remaining: 80.4ms
969:	learn: 4693.4084001	total: 2.52s	remaining: 77.8ms
970:	learn: 4693.0148148	total: 2.52s	remaining: 75.3ms
971:	learn: 4692.1083131	total: 2.52s	remaining: 72.7ms
972:	learn: 4691.1400477	total: 2.52s	remaining: 70.1ms
973:	learn: 4690.6299710	total: 2.53s	remaining: 67.5ms
974:	learn: 4690.1922574	total: 2.53s	remaining: 64.9ms
975:	learn: 4689.5644705	total: 2.53s	remaining: 62.3ms
976:	learn: 4689.2279898	total: 2.54s	remaining: 59.7ms
977:	learn: 4688.7786599	total: 2.54s	remaining: 57.1ms
978:	learn: 4688.1545859	total: 2.54s	remaining: 54.5ms
979:	learn: 4687.5641083	total: 2.54s	remaining: 51.9ms
980:	learn: 4686.9684717	total: 2.55s	remaining: 49.3ms
981:	learn: 4686.3101509	total: 2.55s	remaining: 46.7ms
982:	learn: 4685.6349393	total: 2.55s	remaining: 44.1ms
983:	learn: 4684.9186202	total: 2.55s	remaining: 41.5ms
984:	learn: 4684.1556839	total: 2.56s	remaining: 3

In [8]:
print(model.get_best_score())

{'learn': {'R2': 0.4628500349451464, 'RMSE': 4674.88895862783}}


## Cross validation 

In [9]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
cross_val_score = cross_val_score(model, X_test, y_test, cv=10, scoring='r2').mean()

Learning rate set to 0.06138
0:	learn: 6326.4241977	total: 2.45ms	remaining: 2.45s
1:	learn: 6278.3268680	total: 4.64ms	remaining: 2.32s
2:	learn: 6233.8183915	total: 7.09ms	remaining: 2.36s
3:	learn: 6195.1756927	total: 9.47ms	remaining: 2.36s
4:	learn: 6161.0822501	total: 11.8ms	remaining: 2.36s
5:	learn: 6128.6680833	total: 14.1ms	remaining: 2.34s
6:	learn: 6098.2242337	total: 16.8ms	remaining: 2.38s
7:	learn: 6073.1697376	total: 19.4ms	remaining: 2.4s
8:	learn: 6047.4760093	total: 21.5ms	remaining: 2.37s
9:	learn: 6025.3298418	total: 24.1ms	remaining: 2.39s
10:	learn: 6004.6124173	total: 26.7ms	remaining: 2.4s
11:	learn: 5983.1912458	total: 29.2ms	remaining: 2.4s
12:	learn: 5965.7914212	total: 31.5ms	remaining: 2.39s
13:	learn: 5948.7352256	total: 33.9ms	remaining: 2.39s
14:	learn: 5933.8884164	total: 36.3ms	remaining: 2.39s
15:	learn: 5919.5130862	total: 39.1ms	remaining: 2.4s
16:	learn: 5903.5764316	total: 41.5ms	remaining: 2.4s
17:	learn: 5890.7775895	total: 43.6ms	remaining: 2.

152:	learn: 5497.5087092	total: 388ms	remaining: 2.15s
153:	learn: 5496.1876784	total: 391ms	remaining: 2.15s
154:	learn: 5493.6805656	total: 394ms	remaining: 2.15s
155:	learn: 5491.3553609	total: 397ms	remaining: 2.15s
156:	learn: 5489.1566569	total: 400ms	remaining: 2.15s
157:	learn: 5487.3591342	total: 402ms	remaining: 2.14s
158:	learn: 5486.2022751	total: 405ms	remaining: 2.14s
159:	learn: 5484.9197296	total: 410ms	remaining: 2.15s
160:	learn: 5483.0916279	total: 413ms	remaining: 2.15s
161:	learn: 5481.4782501	total: 415ms	remaining: 2.15s
162:	learn: 5479.8549749	total: 418ms	remaining: 2.14s
163:	learn: 5477.7433162	total: 420ms	remaining: 2.14s
164:	learn: 5476.3864267	total: 423ms	remaining: 2.14s
165:	learn: 5473.1362205	total: 425ms	remaining: 2.13s
166:	learn: 5471.5838495	total: 427ms	remaining: 2.13s
167:	learn: 5470.0808743	total: 429ms	remaining: 2.13s
168:	learn: 5468.7417561	total: 431ms	remaining: 2.12s
169:	learn: 5466.6254171	total: 433ms	remaining: 2.12s
170:	learn

305:	learn: 5270.2667294	total: 768ms	remaining: 1.74s
306:	learn: 5268.9787382	total: 771ms	remaining: 1.74s
307:	learn: 5267.8363553	total: 773ms	remaining: 1.74s
308:	learn: 5266.9945926	total: 776ms	remaining: 1.73s
309:	learn: 5265.7893927	total: 779ms	remaining: 1.73s
310:	learn: 5265.5599902	total: 781ms	remaining: 1.73s
311:	learn: 5264.0479973	total: 783ms	remaining: 1.73s
312:	learn: 5262.9789977	total: 786ms	remaining: 1.73s
313:	learn: 5262.0845508	total: 788ms	remaining: 1.72s
314:	learn: 5260.2731845	total: 791ms	remaining: 1.72s
315:	learn: 5259.1625777	total: 793ms	remaining: 1.72s
316:	learn: 5258.0121205	total: 796ms	remaining: 1.71s
317:	learn: 5256.7955943	total: 798ms	remaining: 1.71s
318:	learn: 5255.9907723	total: 800ms	remaining: 1.71s
319:	learn: 5255.0131762	total: 803ms	remaining: 1.71s
320:	learn: 5254.2060044	total: 805ms	remaining: 1.7s
321:	learn: 5253.2285962	total: 807ms	remaining: 1.7s
322:	learn: 5252.6259844	total: 809ms	remaining: 1.7s
323:	learn: 5

468:	learn: 5102.8202419	total: 1.16s	remaining: 1.31s
469:	learn: 5101.7428240	total: 1.16s	remaining: 1.31s
470:	learn: 5101.0358338	total: 1.17s	remaining: 1.31s
471:	learn: 5099.7951644	total: 1.17s	remaining: 1.31s
472:	learn: 5098.4864592	total: 1.17s	remaining: 1.3s
473:	learn: 5097.6018939	total: 1.17s	remaining: 1.3s
474:	learn: 5096.2949260	total: 1.18s	remaining: 1.3s
475:	learn: 5095.5683987	total: 1.18s	remaining: 1.3s
476:	learn: 5094.6033901	total: 1.18s	remaining: 1.29s
477:	learn: 5093.5798534	total: 1.18s	remaining: 1.29s
478:	learn: 5092.2687404	total: 1.19s	remaining: 1.29s
479:	learn: 5091.3015473	total: 1.19s	remaining: 1.29s
480:	learn: 5091.2137054	total: 1.19s	remaining: 1.28s
481:	learn: 5090.2106650	total: 1.19s	remaining: 1.28s
482:	learn: 5088.9936516	total: 1.2s	remaining: 1.28s
483:	learn: 5087.6423475	total: 1.2s	remaining: 1.28s
484:	learn: 5086.7979296	total: 1.2s	remaining: 1.27s
485:	learn: 5085.7099299	total: 1.2s	remaining: 1.27s
486:	learn: 5084.9

620:	learn: 4958.9820338	total: 1.55s	remaining: 945ms
621:	learn: 4957.8844450	total: 1.55s	remaining: 942ms
622:	learn: 4957.0914692	total: 1.55s	remaining: 940ms
623:	learn: 4956.3292480	total: 1.55s	remaining: 938ms
624:	learn: 4955.6804499	total: 1.56s	remaining: 935ms
625:	learn: 4954.7865275	total: 1.56s	remaining: 933ms
626:	learn: 4953.9754887	total: 1.56s	remaining: 931ms
627:	learn: 4952.8599599	total: 1.57s	remaining: 928ms
628:	learn: 4952.4130248	total: 1.57s	remaining: 926ms
629:	learn: 4951.6809897	total: 1.57s	remaining: 923ms
630:	learn: 4951.1973729	total: 1.57s	remaining: 921ms
631:	learn: 4950.6537076	total: 1.58s	remaining: 918ms
632:	learn: 4949.5663951	total: 1.58s	remaining: 915ms
633:	learn: 4948.5526203	total: 1.58s	remaining: 913ms
634:	learn: 4948.0376066	total: 1.58s	remaining: 910ms
635:	learn: 4947.1923976	total: 1.59s	remaining: 908ms
636:	learn: 4946.5997070	total: 1.59s	remaining: 905ms
637:	learn: 4945.4907850	total: 1.59s	remaining: 903ms
638:	learn

...

13:	learn: 6028.5075354	total: 23.9ms	remaining: 1.68s
14:	learn: 6010.3388959	total: 25.9ms	remaining: 1.7s
15:	learn: 5993.5769206	total: 27.8ms	remaining: 1.71s
16:	learn: 5976.7936928	total: 29.4ms	remaining: 1.7s
17:	learn: 5961.5077816	total: 30.8ms	remaining: 1.68s
18:	learn: 5950.8182336	total: 32.5ms	remaining: 1.68s
19:	learn: 5937.9990309	total: 34.1ms	remaining: 1.67s
20:	learn: 5927.4148967	total: 35.5ms	remaining: 1.66s
21:	learn: 5916.6843042	total: 37.3ms	remaining: 1.66s
22:	learn: 5905.9683601	total: 39.4ms	remaining: 1.67s
23:	learn: 5895.1084600	total: 41.2ms	remaining: 1.68s
24:	learn: 5883.4958638	total: 43ms	remaining: 1.68s
25:	learn: 5875.1267505	total: 44.6ms	remaining: 1.67s
26:	learn: 5864.3906672	total: 46.3ms	remaining: 1.67s
27:	learn: 5855.6033941	total: 48.1ms	remaining: 1.67s
28:	learn: 5847.7855066	total: 49.8ms	remaining: 1.67s
29:	learn: 5839.2148821	total: 51.8ms	remaining: 1.67s
30:	learn: 5830.9635631	total: 53.7ms	remaining: 1.68s
31:	learn: 582

233:	learn: 5201.6083455	total: 412ms	remaining: 1.35s
234:	learn: 5199.8646963	total: 414ms	remaining: 1.35s
235:	learn: 5196.9514584	total: 416ms	remaining: 1.34s
236:	learn: 5194.5336627	total: 417ms	remaining: 1.34s
237:	learn: 5192.5813910	total: 419ms	remaining: 1.34s
238:	learn: 5191.0619386	total: 421ms	remaining: 1.34s
239:	learn: 5187.3523229	total: 423ms	remaining: 1.34s
240:	learn: 5184.5769831	total: 425ms	remaining: 1.34s
241:	learn: 5182.6061587	total: 426ms	remaining: 1.33s
242:	learn: 5181.3867320	total: 428ms	remaining: 1.33s
243:	learn: 5180.0201638	total: 430ms	remaining: 1.33s
244:	learn: 5177.4910089	total: 432ms	remaining: 1.33s
245:	learn: 5175.1444740	total: 434ms	remaining: 1.33s
246:	learn: 5173.0568865	total: 435ms	remaining: 1.33s
247:	learn: 5170.8183743	total: 437ms	remaining: 1.32s
248:	learn: 5168.6794478	total: 439ms	remaining: 1.32s
249:	learn: 5166.6216291	total: 441ms	remaining: 1.32s
250:	learn: 5163.7179259	total: 443ms	remaining: 1.32s
251:	learn

448:	learn: 4772.8703523	total: 797ms	remaining: 978ms
449:	learn: 4771.5291950	total: 799ms	remaining: 977ms
450:	learn: 4770.2077403	total: 801ms	remaining: 975ms
451:	learn: 4768.5229159	total: 802ms	remaining: 973ms
452:	learn: 4767.3072261	total: 804ms	remaining: 971ms
453:	learn: 4765.0305221	total: 806ms	remaining: 969ms
454:	learn: 4762.1780289	total: 808ms	remaining: 968ms
455:	learn: 4760.7854946	total: 810ms	remaining: 966ms
456:	learn: 4759.5029282	total: 812ms	remaining: 965ms
457:	learn: 4758.2036507	total: 814ms	remaining: 963ms
458:	learn: 4756.9221256	total: 816ms	remaining: 962ms
459:	learn: 4755.9115803	total: 818ms	remaining: 960ms
460:	learn: 4755.1480558	total: 819ms	remaining: 958ms
461:	learn: 4752.9844675	total: 821ms	remaining: 956ms
462:	learn: 4750.9035217	total: 823ms	remaining: 955ms
463:	learn: 4748.9392719	total: 825ms	remaining: 953ms
464:	learn: 4746.6682599	total: 827ms	remaining: 951ms
465:	learn: 4744.8887117	total: 829ms	remaining: 950ms
466:	learn

663:	learn: 4440.8275608	total: 1.18s	remaining: 599ms
664:	learn: 4439.6449277	total: 1.19s	remaining: 597ms
665:	learn: 4438.4772030	total: 1.19s	remaining: 595ms
666:	learn: 4437.3037701	total: 1.19s	remaining: 593ms
667:	learn: 4435.8698085	total: 1.19s	remaining: 591ms
668:	learn: 4434.2134689	total: 1.19s	remaining: 590ms
669:	learn: 4433.0440999	total: 1.19s	remaining: 588ms
670:	learn: 4432.0454653	total: 1.2s	remaining: 587ms
671:	learn: 4431.1438083	total: 1.2s	remaining: 585ms
672:	learn: 4430.3394790	total: 1.2s	remaining: 583ms
673:	learn: 4429.6302746	total: 1.2s	remaining: 581ms
674:	learn: 4428.9694096	total: 1.2s	remaining: 580ms
675:	learn: 4427.7084302	total: 1.21s	remaining: 578ms
676:	learn: 4425.9016695	total: 1.21s	remaining: 576ms
677:	learn: 4424.0855529	total: 1.21s	remaining: 574ms
678:	learn: 4423.0802063	total: 1.21s	remaining: 573ms
679:	learn: 4421.6720648	total: 1.21s	remaining: 571ms
680:	learn: 4419.8327509	total: 1.22s	remaining: 569ms
681:	learn: 441

869:	learn: 4197.4115310	total: 1.55s	remaining: 232ms
870:	learn: 4195.9266341	total: 1.55s	remaining: 230ms
871:	learn: 4195.1372963	total: 1.55s	remaining: 228ms
872:	learn: 4193.5375829	total: 1.56s	remaining: 226ms
873:	learn: 4192.4104914	total: 1.56s	remaining: 225ms
874:	learn: 4190.9490868	total: 1.56s	remaining: 223ms
875:	learn: 4190.4330183	total: 1.56s	remaining: 221ms
876:	learn: 4189.1911054	total: 1.56s	remaining: 219ms
877:	learn: 4187.7915628	total: 1.56s	remaining: 218ms
878:	learn: 4187.2262217	total: 1.57s	remaining: 216ms
879:	learn: 4185.9690683	total: 1.57s	remaining: 214ms
880:	learn: 4185.1973335	total: 1.57s	remaining: 212ms
881:	learn: 4184.5565872	total: 1.57s	remaining: 210ms
882:	learn: 4183.6372892	total: 1.57s	remaining: 209ms
883:	learn: 4182.2495081	total: 1.58s	remaining: 207ms
884:	learn: 4180.6397828	total: 1.58s	remaining: 205ms
885:	learn: 4179.3568338	total: 1.58s	remaining: 203ms
886:	learn: 4178.0974509	total: 1.58s	remaining: 202ms
887:	learn

In [10]:
cross_val_score

0.17678859765276064